In [ ]:
import sys
sys.path.append("..")
import rasterio
import geopandas as gpd
import utils.data as dt
import csv
import numpy as np
import shutil
from pathlib import Path

The three parameters of this script are,

* Which directory contains all the scenes that we want to extract preprocessing-relevant summaries and masks for?
* Where do we want to save our results?
* What is the full path to the labels from which we will construct our masks?

For the first question, we will use the following choices (on the Azure machine),

* Bing Recent: `/datadrive/glaciers/bing_glaciers/bing_glacial_lakes`
* Landsat 2015: `/datadrive/snake/lakes/le7-2015/{split}`
* Landsat all: `/datadrive/snake/lakes/imagery`

Technically we only need Landsat all, but Landsat 2015 is convenient because that's the only data that we can train on (the rest would be for purely inference purposes).

In [ ]:
in_dir = Path("/datadrive/snake/lakes/le7-2015/splits/train")
out_dir = Path("/datadrive/snake/lakes/le7-2015/splits/train/processed")
label_path = Path("/datadrive/snake/lakes/GL_3basins_2015.shp")

if out_dir.exists():
    shutil.rmtree(out_dir)
out_dir.mkdir(parents=True)

Next, we read in the label data and set up the writer to which we will save summary statistics.

In [ ]:
scene_list = in_dir.glob("*.tif")
y = gpd.read_file(label_path)
fields = ["scene"] + sum([[f"{s}_{i}" for i in range(11)] for s in ["mean", "sdev"]], [])
stat_path = out_dir / "statistics.csv"
f = open(stat_path, "a")
writer = csv.writer(f)
writer.writerow(fields)

Finally, we can loop over all the scenes in `in_dir` and save the relevant statistics and masks.

In [ ]:
for scene in scene_list:
    img = rasterio.open(scene)
    result = dt.preprocessor(img, y)

    out_meta = img.meta
    out_meta.update({
        "driver": "GTiff", 
        "height": result[2].shape[1], 
        "width": result[2].shape[2],
        "count": result[2].shape[0],
        "transform": img.transform
    })

    with rasterio.open(out_dir / f"{scene.stem}-labels.tif", "w", **out_meta) as f:
        f.write(result[2])
    writer.writerow([str(scene)] + list(np.hstack(result[:2])))
    
f.close()

In [ ]:
import matplotlib.pyplot as plt
for s in result[2]:
    plt.imshow(s)
    plt.show()